In [15]:
import pandas as pd
import yfinance as yf
import talib
import pytz
from backtesting import Backtest, Strategy

TRADING_START = (9, 15)
TRADING_END = (15, 30)
LAST_ENTRY = (15, 25)  # Last possible entry (so you can force-exit in next 5-min candle)

class ScalpingStrategy(Strategy):
    ma_period = 5
    ema_period = 9
    bb_period = 20
    bb_std_dev = 2
    trailing_sl_pct = 0.02

    def init(self):
        self.ma = self.I(talib.SMA, self.data.Close, self.ma_period)
        self.ema = self.I(talib.EMA, self.data.Close, self.ema_period)
        bb = self.I(talib.BBANDS, self.data.Close, self.bb_period, self.bb_std_dev, self.bb_std_dev, 0)
        self.upper_band, self.middle_band, self.lower_band = bb[0], bb[1], bb[2]
        self.long_peak = None
        self.short_trough = None

    def next(self):
        price = self.data.Close[-1]
        current_dt = self.data.index[-1]
        hour, minute = current_dt.hour, current_dt.minute

        market_open = (hour > TRADING_START[0] or (hour == TRADING_START[0] and minute >= TRADING_START[1]))
        before_last_bar = (hour < LAST_ENTRY[0]) or (hour == LAST_ENTRY[0] and minute <= LAST_ENTRY[1])
        within_day = market_open and ((hour < TRADING_END[0]) or (hour == TRADING_END[0] and minute <= TRADING_END[1]))

        # --- Force-close at or after 15:25 (before or at 15:30 no matter what) ---
        if self.position:
            # If after or at 15:25, close position forcibly at every bar
            if hour > LAST_ENTRY[0] or (hour == LAST_ENTRY[0] and minute >= LAST_ENTRY[1]):
                self.position.close()
                self.long_peak = None
                self.short_trough = None
                return

        # --- Trailing stops (only intraday) ---
        if self.position and within_day:
            if self.position.is_long:
                self.long_peak = price if self.long_peak is None else max(self.long_peak, price)
                sl = self.long_peak * (1 - self.trailing_sl_pct)
                if price <= sl:
                    self.position.close()
                    self.long_peak = None
            elif self.position.is_short:
                self.short_trough = price if self.short_trough is None else min(self.short_trough, price)
                sl = self.short_trough * (1 + self.trailing_sl_pct)
                if price >= sl:
                    self.position.close()
                    self.short_trough = None

        # --- Entry logic: ONLY allow entry between 9:15 and 15:25 ---
        if not self.position and market_open and before_last_bar and within_day:
            if (price > self.ma[-1] and self.ma[-1] > self.ema[-1] and price > self.upper_band[-1]):
                self.buy()
                self.long_peak = price
                self.short_trough = None
            elif (price < self.ma[-1] and self.ma[-1] < self.ema[-1] and price < self.lower_band[-1]):
                self.sell()
                self.short_trough = price
                self.long_peak = None

if __name__ == '__main__':
    data = yf.download('RELIANCE.NS', period='60d', interval='5m', multi_level_index=False)
    data.index = data.index.tz_convert('Asia/Kolkata')

    # Filter ONLY rows during market hours (for clarity in plotting & export)
    is_market_time = (
        (data.index.hour > 9) | ((data.index.hour == 9) & (data.index.minute >= 15))
    ) & (
        (data.index.hour < 15) | ((data.index.hour == 15) & (data.index.minute <= 30))
    )
    data = data.loc[is_market_time]

    if data.empty:
        print("No data downloaded. Please check the ticker symbol or data provider.")
    else:
        bt = Backtest(data, ScalpingStrategy, cash=100000, commission=0.002)
        stats = bt.run()

        print('\n--- Backtest Metrics (Intraday, NSE 9:15–15:30 IST only) ---')
        print(stats)

        trades_df = stats['_trades']
        print('\n--- Trades Taken ---')
        print(trades_df)

        # Save trades to CSV
        trades_df.to_csv('scalping_strategy_intraday_trades_ist_strict.csv', index=False)
        print("\nTrade details saved to 'scalping_strategy_intraday_trades_ist_strict.csv'")

        bt.plot()


/var/folders/mn/tx231v5158797chnp22wm9000000gp/T/ipykernel_29367/1197422432.py:71: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download('RELIANCE.NS', period='60d', interval='5m', multi_level_index=False)
[*********************100%***********************]  1 of 1 completed


Backtest.run:   0%|          | 0/4480 [00:00<?, ?bar/s]

/var/folders/mn/tx231v5158797chnp22wm9000000gp/T/ipykernel_29367/1197422432.py:86: UserWarning: Some trades remain open at the end of backtest. Use `Backtest(..., finalize_trades=True)` to close them and include them in stats.
  stats = bt.run()



--- Backtest Metrics (Intraday, NSE 9:15–15:30 IST only) ---
Start                     2025-05-12 09:15...
End                       2025-08-01 15:25...
Duration                     81 days 06:10:00
Exposure Time [%]                        85.4
Equity Final [$]                    81790.026
Equity Peak [$]                  100682.49829
Commissions [$]                   21471.63899
Return [%]                          -18.20997
Buy & Hold Return [%]                -1.94824
Return (Ann.) [%]                   -58.08879
Volatility (Ann.) [%]                 6.42742
CAGR [%]                            -46.38826
Sharpe Ratio                         -9.03765
Sortino Ratio                        -4.11162
Calmar Ratio                         -3.07118
Alpha [%]                           -17.94887
Beta                                  0.13402
Max. Drawdown [%]                   -18.91418
Avg. Drawdown [%]                    -3.90654
Max. Drawdown Duration       81 days 00:05:00
Avg. Drawdown Dura

/Users/hemank/Documents/github/.talib/lib/python3.12/site-packages/bokeh/util/serialization.py:242: UserWarning: no explicit representation of timezones available for np.datetime64
  return convert(array.astype("datetime64[us]"))


In [66]:
import pandas as pd
import numpy as np
import yfinance as yf
import talib
from datetime import time
import pytz


# PARAMETERS
symbol = "SUZLON.NS"
capital = 100000         # ₹1,00,000 starting capital
leverage = 5
atr_period = 14
atr_mult = 2            # ATR multiplier for stop loss
intraday_start = time(9, 15)
intraday_end = time(15, 30)
force_exit_time = time(15, 0)
ist = pytz.timezone('Asia/Kolkata')


# Download 5-minute data (~60 days)
data = yf.download(symbol, interval='5m', period='60d', progress=False, prepost=False, auto_adjust=True, multi_level_index=False)
data = data.tz_convert('Asia/Kolkata')
data = data.between_time(intraday_start, intraday_end).dropna()


# Calculate indicators
data['SMA5'] = talib.SMA(data['Close'], timeperiod=5)
data['EMA9'] = talib.EMA(data['Close'], timeperiod=9)
bb_upper, bb_middle, bb_lower = talib.BBANDS(data['Close'], timeperiod=20, nbdevup=2, nbdevdn=2)
data['BB_UPPER'] = bb_upper
data['BB_LOWER'] = bb_lower
data['ATR'] = talib.ATR(data['High'], data['Low'], data['Close'], timeperiod=atr_period)


# Helper functions for brokerage/costs
def brokerage(order_value):
    raw_brokerage = min(20, order_value * 0.001)
    if raw_brokerage < 5:
        return min(5, order_value * 0.025)
    return raw_brokerage


def stt(trade_val): return trade_val * 0.00025   # 0.025% on sell side only
def stamp_duty(trade_val): return trade_val * 0.00003   # 0.003% on buy side only
def exchange_charges(trade_val): return trade_val * 0.0000297   # 0.00297% both sides (NSE rate)
def sebi_charges(trade_val): return trade_val * 0.000001   # 0.0001% both sides
def ipf_charges(trade_val): return trade_val * 0.000001   # 0.0001% both sides
def gst(broker, exch, sebi): return 0.18 * (broker + exch + sebi)


# Trade simulation state
trades = []
position = None
stop_loss = None
max_price = None
min_price = None


for i in range(1, len(data)):
    row = data.iloc[i]
    prev = data.iloc[i-1]
    ts = row.name


    # Force exit before 15:00 IST
    if ts.time() >= force_exit_time:
        if position:
            exit_price = row['Close']
            exit_time_pos = ts
            qty = position['qty']

            entry_val = position['entry_price'] * qty
            exit_val = exit_price * qty

            # PnL calculation
            if position['direction'] == 'long':
                gross_pnl = (exit_price - position['entry_price']) * qty
                stt_ = 0                            # No STT on buy
                stamp = stamp_duty(entry_val)      # Stamp duty on buy side (entry)
            else:  # short
                gross_pnl = (position['entry_price'] - exit_price) * qty
                stt_ = stt(entry_val)              # STT on sell side (entry)
                stamp = stamp_duty(exit_val)       # Stamp duty on buy side (exit)

            # Charges
            broker = brokerage(entry_val) + brokerage(exit_val)
            exch = exchange_charges(entry_val) + exchange_charges(exit_val)
            sebi_ = sebi_charges(entry_val) + sebi_charges(exit_val)
            ipf_ = ipf_charges(entry_val) + ipf_charges(exit_val)
            gst_ = gst(broker, exch, sebi_)
            total_cost = broker + exch + sebi_ + ipf_ + stt_ + stamp + gst_

            net_pnl = gross_pnl - total_cost

            trades.append({
                'entry_time': position['entry_time'],
                'exit_time': exit_time_pos,
                'direction': position['direction'],
                'entry_price': position['entry_price'],
                'exit_price': exit_price,
                'qty': qty,
                'gross_pnl': gross_pnl,
                'net_pnl': net_pnl,
                'costs': total_cost,
            
                'brokerage': broker,
                'exchange_charges': exch,
                'sebi_charges': sebi_,
                'ipf_charges': ipf_,
                'stt': stt_,
                'stamp': stamp,
                'gst': gst_
            })
            position = None
        continue


    # Only consider bars with valid ATR and indicators
    if pd.isna(row['ATR']) or pd.isna(row['SMA5']) or pd.isna(row['EMA9']) or pd.isna(row['BB_UPPER']) or pd.isna(row['BB_LOWER']):
        continue


    # Entry signals
    long_signal = (prev['Close'] < prev['SMA5'] and prev['Close'] < prev['EMA9'] and
                   row['Close'] > row['SMA5'] and row['Close'] > row['EMA9'] and
                   row['Close'] > row['BB_LOWER'])
    short_signal = (prev['Close'] > prev['SMA5'] and prev['Close'] > prev['EMA9'] and
                    row['Close'] < row['SMA5'] and row['Close'] < row['EMA9'] and
                    row['Close'] < row['BB_UPPER'])


    price = row['Close']


    if position is None:
        max_cap = capital * leverage
        qty = int(max_cap // price)
        if qty == 0:
            continue
        if long_signal:
            position = {'entry_time': ts, 'entry_price': price, 'qty': qty, 'direction': 'long'}
            stop_loss = price - atr_mult * row['ATR']  # ATR-based stop loss
            max_price = price
        elif short_signal:
            position = {'entry_time': ts, 'entry_price': price, 'qty': qty, 'direction': 'short'}
            stop_loss = price + atr_mult * row['ATR']
            min_price = price


    elif position['direction'] == 'long':
        max_price = max(max_price, price)
        # Trail stop loss: highest price - ATR * multiplier
        new_stop = max_price - atr_mult * row['ATR']
        stop_loss = max(stop_loss, new_stop)
        if price <= stop_loss or ts.time() >= force_exit_time:
            exit_price = price
            exit_time_pos = ts
            qty = position['qty']

            entry_val = position['entry_price'] * qty
            exit_val = exit_price * qty

            gross_pnl = (exit_price - position['entry_price']) * qty
            stt_ = 0
            stamp = stamp_duty(entry_val)

            broker = brokerage(entry_val) + brokerage(exit_val)
            exch = exchange_charges(entry_val) + exchange_charges(exit_val)
            sebi_ = sebi_charges(entry_val) + sebi_charges(exit_val)
            ipf_ = ipf_charges(entry_val) + ipf_charges(exit_val)
            gst_ = gst(broker, exch, sebi_)

            total_cost = broker + exch + sebi_ + ipf_ + stt_ + stamp + gst_

            net_pnl = gross_pnl - total_cost

            trades.append({
                'entry_time': position['entry_time'],
                'exit_time': exit_time_pos,
                'direction': 'long',
                'entry_price': position['entry_price'],
                'exit_price': exit_price,
                'qty': qty,
                'gross_pnl': gross_pnl,
                'net_pnl': net_pnl,
                'costs': total_cost,

                'brokerage': broker,
                'exchange_charges': exch,
                'sebi_charges': sebi_,
                'ipf_charges': ipf_,
                'stt': stt_,
                'stamp': stamp,
                'gst': gst_
            })
            position = None


    elif position['direction'] == 'short':
        min_price = min(min_price, price)
        # Trail stop loss: lowest price + ATR * multiplier
        new_stop = min_price + atr_mult * row['ATR']
        stop_loss = min(stop_loss, new_stop)
        if price >= stop_loss or ts.time() >= force_exit_time:
            exit_price = price
            exit_time_pos = ts
            qty = position['qty']

            entry_val = position['entry_price'] * qty
            exit_val = price * qty

            gross_pnl = (position['entry_price'] - exit_price) * qty
            stt_ = stt(entry_val)
            stamp = stamp_duty(exit_val)

            broker = brokerage(entry_val) + brokerage(exit_val)
            exch = exchange_charges(entry_val) + exchange_charges(exit_val)
            sebi_ = sebi_charges(entry_val) + sebi_charges(exit_val)
            ipf_ = ipf_charges(entry_val) + ipf_charges(exit_val)
            gst_ = gst(broker, exch, sebi_)

            total_cost = broker + exch + sebi_ + ipf_ + stt_ + stamp + gst_

            net_pnl = gross_pnl - total_cost

            trades.append({
                'entry_time': position['entry_time'],
                'exit_time': exit_time_pos,
                'direction': 'short',
                'entry_price': position['entry_price'],
                'exit_price': exit_price,
                'qty': qty,
                'gross_pnl': gross_pnl,
                'net_pnl': net_pnl,
                'costs': total_cost,

                'brokerage': broker,
                'exchange_charges': exch,
                'sebi_charges': sebi_,
                'ipf_charges': ipf_,
                'stt': stt_,
                'stamp': stamp,
                'gst': gst_
            })
            position = None


# Export trades to CSV
trades_df = pd.DataFrame(trades)
trades_df.to_csv('intraday_scalping_trades_atr_stop.csv', index=False)


# Performance metrics
net_pnl = trades_df['net_pnl'].sum()
win_trades = (trades_df['net_pnl'] > 0).sum()
total_trades = len(trades_df)
win_rate = (win_trades / total_trades) * 100 if total_trades > 0 else 0
avg_trade_pnl = trades_df['net_pnl'].mean() if total_trades > 0 else 0
cum_pnl = trades_df['net_pnl'].cumsum()
max_drawdown = (cum_pnl.cummax() - cum_pnl).max() if total_trades > 0 else 0
total_costs = trades_df['costs'].sum()


print("Final Performance Metrics:")
print(f"Net PnL: ₹{net_pnl:,.2f}")
print(f"Win rate: {win_rate:.2f}%")
print(f"Avg trade PnL: ₹{avg_trade_pnl:,.2f}")
print(f"Max drawdown: ₹{max_drawdown:,.2f}")
print(f"Number of trades: {total_trades}")
print(f"Total brokerage/charges paid: ₹{total_costs:,.2f}")
print("Trade details exported to 'intraday_scalping_trades_atr_stop.csv'")


Final Performance Metrics:
Net PnL: ₹-71,472.13
Win rate: 33.75%
Avg trade PnL: ₹-446.70
Max drawdown: ₹117,133.50
Number of trades: 160
Total brokerage/charges paid: ₹26,282.42
Trade details exported to 'intraday_scalping_trades_atr_stop.csv'
